In [ ]:
# Kiểm tra kích thước trước khi xóa
print(f"Kích thước ban đầu: {df.shape}")

# Loại bỏ các dữ liệu bị chặn trần (Capped Data)
# Chỉ giữ lại các hàng có giá nhà < 500,000
df_clean = df[df['median_house_value'] < 500000].copy()

# Kiểm tra lại
print(f"Kích thước sau khi loại bỏ outlier: {df_clean.shape}")
print(f"Đã loại bỏ {df.shape[0] - df_clean.shape[0]} hàng nhiễu.")

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 1. Trước khi lọc (Dữ liệu gốc có trong df) - Minh họa lại
# Lưu ý: Bạn cần chắc chắn biến 'df' gốc chưa bị ghi đè, hoặc vẽ cái này trước khi chạy lệnh lọc
sns.histplot(df['median_house_value'], bins=50, kde=True, ax=axes[0], color='red')
axes[0].set_title('TRƯỚC: Dữ liệu bị chặn trần (Capping) tại 500k', fontsize=12, fontweight='bold')
axes[0].axvline(500000, color='black', linestyle='--') # Đường kẻ chỉ điểm lỗi

# 2. Sau khi lọc (Dữ liệu trong df_clean)
sns.histplot(df_clean['median_house_value'], bins=50, kde=True, ax=axes[1], color='green')
axes[1].set_title('SAU: Dữ liệu đã sạch (Loại bỏ nhiễu)', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

# Tính toán giá trị trung vị của cột total_bedrooms
median_bedrooms = df_clean['total_bedrooms'].median()

# Điền giá trị thiếu
df_clean['total_bedrooms'] = df_clean['total_bedrooms'].fillna(median_bedrooms)

# Kiểm tra lại xem còn null không
print(f"Số lượng giá trị thiếu sau khi xử lý: {df_clean['total_bedrooms'].isnull().sum()}")

# Sử dụng Pandas get_dummies để One-Hot Encoding
df_clean = pd.get_dummies(df_clean, columns=['ocean_proximity'], dtype=int)

# Xem 5 dòng đầu sau khi mã hóa
df_clean.head()

# Tạo các đặc trưng mới (tỷ lệ)
df_clean['rooms_per_household'] = df_clean['total_rooms'] / df_clean['households']
df_clean['bedrooms_per_room'] = df_clean['total_bedrooms'] / df_clean['total_rooms']
df_clean['population_per_household'] = df_clean['population'] / df_clean['households']

# Kiểm tra tương quan của biến mới với giá nhà
new_features = ['rooms_per_household', 'bedrooms_per_room', 'population_per_household', 'median_house_value']
print(df_clean[new_features].corr()['median_house_value'])

# Tính độ tương quan
corr_old = df['total_rooms'].corr(df['median_house_value'])
corr_new = df_clean['rooms_per_household'].corr(df_clean['median_house_value'])

# Vẽ biểu đồ so sánh
plt.figure(figsize=(8, 5))
bars = plt.bar(['Total Rooms (Cũ)', 'Rooms/Household (Mới)'], [corr_old, corr_new], color=['gray', 'blue'])

plt.title('Hiệu quả của Feature Engineering', fontsize=14)
plt.ylabel('Hệ số Tương quan với Giá nhà')
plt.axhline(0, color='black', linewidth=0.5)

# Hiển thị con số cụ thể trên cột
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height,
             f'{height:.3f}',
             ha='center', va='bottom', fontsize=12, fontweight='bold')

plt.show()

from sklearn.model_selection import train_test_split

# 1. Tách biến mục tiêu (y) và đặc trưng (X)
X = df_clean.drop("median_house_value", axis=1) # X là tất cả trừ giá nhà
y = df_clean["median_house_value"]              # y là giá nhà

# 2. Chia tập dữ liệu: 80% Train, 20% Test
# random_state=42 giúp cố định kết quả để tái hiện sau này
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Kích thước tập Huấn luyện: {X_train.shape}")
print(f"Kích thước tập Kiểm tra: {X_test.shape}")

from sklearn.preprocessing import StandardScaler

# Khởi tạo bộ chuẩn hóa
scaler = StandardScaler()

# Fit và Transform trên tập Train
X_train_scaled = scaler.fit_transform(X_train)

# Chỉ Transform trên tập Test (không Fit lại!)
X_test_scaled = scaler.transform(X_test)

# Chuyển đổi lại thành DataFrame để dễ theo dõi (tùy chọn, nhưng tốt cho minh bạch)
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X.columns, index=X_train.index)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X.columns, index=X_test.index)

print("Đã chuẩn hóa xong. Dữ liệu sẵn sàng cho Model.")
X_train_scaled.head(3)

# Chuyển X_train (chưa scale) thành DataFrame tạm để vẽ (nếu cần)
# Ở cell 5 bạn đã có X_train rồi.

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# 1. Trước khi Scale (Dữ liệu Thô)
# So sánh Thu nhập (số bé) và Dân số (số to)
ax1.scatter(X_train['median_income'], X_train['population'], alpha=0.5, color='orange')
ax1.set_title('TRƯỚC: Thang đo chênh lệch lớn', fontsize=12, fontweight='bold')
ax1.set_xlabel('Thu nhập (0 - 15)')
ax1.set_ylabel('Dân số (0 - 35,000)')

# 2. Sau khi Scale (Dữ liệu Chuẩn hóa)
# Dùng X_train_scaled
ax2.scatter(X_train_scaled['median_income'], X_train_scaled['population'], alpha=0.5, color='purple')
ax2.set_title('SAU: Dữ liệu đã chuẩn hóa (StandardScaler)', fontsize=12, fontweight='bold')
ax2.set_xlabel('Thu nhập (Std)')
ax2.set_ylabel('Dân số (Std)')

plt.suptitle('Tại sao cần Scaling? (Đưa về cùng hệ quy chiếu)', fontsize=15)
plt.show()

